In [1]:
from nrs import NRS_Model, Element, NRS_Revision, NRS_Observer_E    # классы НРС
from nrs import NRS_Data as nd                                      # Табличные данные НРС
from nrs import q_out_simple, q_out_nozzle                          # модели расчета параметров
import matplotlib.pyplot as plt                                     # библиотеки для отрисовки получаемых данных

# Простая схема с перекачкой
![Схема НРС](imgs/pump_to_pump_2_nozzles.png)

Схема НРС состоит из:
* 2 источников в виде насоса ПА, соединенных последовательно
* 1 магистральной рукавной линии
* 2 рабочих рукавных линий
* 2 приборов подачи воды

In [54]:
# Вычисляем проводимость насадка для ствола Б
p_nozzleB = NRS_Revision.calc_p(3.7, 40)
p_nozzleB

# Создаем элементы НРС
pump1 = Element('Н1', e_type=0, H_add=20)                                 # Стартовый напор на насосе = 20м    
pump2 = Element('Н2', e_type=1, H_add=20)

hoseM1 = Element('МРЛ1', e_type=1, s=nd.ss["77"], n=4)
hoseW1 = Element('РРЛ1', e_type=1, s=nd.ss["51"], n=3)
hoseW2 = Element('РРЛ2', e_type=1, s=nd.ss["51"], n=3)

nozzle1 = Element('Ств1', e_type=2, p=p_nozzleB, q_out = q_out_nozzle)
nozzle2 = Element('Ств2', e_type=2, p=p_nozzleB, q_out = q_out_nozzle)

# Соединяем элементы друг с другом:
pump1.append(hoseM1).append(pump2)
pump2.append(hoseW1).append(nozzle1)
pump2.append(hoseW2).append(nozzle2)

# Без наблюдателей

# Строим модель и инициируем обозревателей
model = NRS_Model('Простая модель от одного насоса к двум стволам, с перекачкой')
# model.build(pump1, interpretate=False);
model.build(pump1, interpretate=True)

# Печатаем состав модели
NRS_Revision.print_model_elements(model)

Новый элемент НРС: Н1
Новый элемент НРС: Н2
Новый элемент НРС: МРЛ1
Новый элемент НРС: РРЛ1
Новый элемент НРС: РРЛ2
Новый элемент НРС: Ств1
Новый элемент НРС: Ств2
Новая модель: Простая модель от одного насоса к двум стволам, с перекачкой
all:
  Н1
  МРЛ1
  Н2
  РРЛ1
  Ств1
  РРЛ2
  Ств2
in:
  Н1
out:
  Ств1
  Ств2


In [55]:
# Рассчет 
model.calc(accuracy=0.05)
print(model.summaryQ())

Расчет НРС не возможен
6.716139980210957


In [56]:
NRS_Revision.print_element_state(pump2)

elements_next: [<nrs.Element object at 0x000002BC1178BC20>, <nrs.Element object at 0x000002BC119DA480>]
elements_previous: [<nrs.Element object at 0x000002BC119D8920>]
type: 1
name: Н2
q: 6.716139980210957
s: 0
H_in: 17.313725342457275
h: 0.0
z: 0
p: 1
n: 1
q_out: <function q_out_simple at 0x000002BC10628400>
H_add: 20
observer: None
H_out: 37.313725342457275


In [57]:
pump2.set_H_add(40)
hoseM1.n = 5
model.calc(accuracy=0.005)
print(model.summaryQ())
hoseM1.n = 8
model.calc(accuracy=0.005)
print(model.summaryQ())
hoseM1.n = 11
model.calc(accuracy=0.005)
print(model.summaryQ())
hoseM1.n = 14
model.calc(accuracy=0.005)
print(model.summaryQ())
hoseM1.n = 17
model.calc(accuracy=0.005)
print(model.summaryQ())

8.151344100978687
7.9552764308168795
7.772273556690172
7.6039574750625
7.44237673472469


In [62]:
hoseM1.n = 2
model.calc(accuracy=0.005)
print(model.summaryQ())

Расчет НРС не возможен
9.656178472499992


In [61]:
# Исправляет перегрев модели, сбрасывая входящие напоры. Для много стартовых моделей неприемлемо, скорее всего.
pump1.set_H_in(20)

In [52]:
model.build(pump1, interpretate=True)

# Схема со слиянием нескольких элементов в одну линию

![Схема НРС](imgs/2_pump_to_1_nozzle.png)

Схема НРС состоит из:
* 2 источников в виде насоса ПА
* 2 магистральных рукавных линий
* 1 рабочей рукавной линии
* 1 прибора подачи воды

In [5]:
# Вычисляем проводимость насадка для ствола Б
p_nozzleL = NRS_Revision.calc_p(20, 60)
print(f'Проводимость лафетного ствола {p_nozzleL}')

# Создаем элементы НРС
pump1 = Element('Н1', e_type=0, H_add=80)                                 
pump2 = Element('Н2', e_type=0, H_add=80)

hoseM1 = Element('МРЛ1', e_type=1, s=nd.ss["77"], n=8)
hoseM2 = Element('МРЛ2', e_type=1, s=nd.ss["77"], n=4)
hoseW1 = Element('РРЛ1', e_type=1, s=nd.ss["77"], n=4)

joiner1 = Element('ВС1', 1)

nozzle1 = Element('Ств1', e_type=2, p=p_nozzleL, q_out = q_out_nozzle)

# Соединяем элементы друг с другом:
pump1.append(hoseM1).append(joiner1)
pump2.append(hoseM2).append(joiner1).append(hoseW1).append(nozzle1)

# Без наблюдателей

# Строим модель и инициируем обозревателей
model = NRS_Model('Сложная модель со слиянием нескольких элементов в одну линию')
# model.build(pump1, interpretate=False);
model.build(pump1, interpretate=True)

# Печатаем состав модели
NRS_Revision.print_model_elements(model)

Проводимость лафетного ствола 2.581988897471611
Новый элемент НРС: Н1
Новый элемент НРС: Н2
Новый элемент НРС: МРЛ1
Новый элемент НРС: МРЛ2
Новый элемент НРС: РРЛ1
Новый элемент НРС: ВС1
Новый элемент НРС: Ств1
Новая модель: Сложная модель со слиянием нескольких элементов в одну линию
all:
  Н1
  МРЛ1
  ВС1
  РРЛ1
  Ств1
  МРЛ2
  Н2
in:
  Н1
  Н2
out:
  Ств1


In [6]:
model.calc(accuracy=0.005)
print(model.summaryQ())

Расчет НРС не возможен
18.857260790083174


In [8]:
print('Насос №1')
NRS_Revision.print_element_state(pump1, ['H_add', 'q'])
print('Насос №2')
NRS_Revision.print_element_state(pump2, ['H_add', 'q'])

Насос №1
H_add: 80
q: 9.428630395041587
Насос №2
H_add: 80
q: 9.428630395041587


In [9]:
NRS_Revision.print_element_state(hoseM1)

elements_next: [<nrs.Element object at 0x00000218FC51DC10>]
elements_previous: [<nrs.Element object at 0x00000218FC46C830>]
type: 1
name: МРЛ1
q: 9.428630395041587
s: 0.015
H_in: 80.0
h: 10.6642229296875
z: 0
p: 1
n: 8
q_out: <function q_out_simple at 0x00000218FB2A5760>
H_add: 0
observer: None
H_out: 69.3357770703125


In [10]:
NRS_Revision.print_element_state(hoseM2)

elements_next: [<nrs.Element object at 0x00000218FC51DC10>]
elements_previous: [<nrs.Element object at 0x00000218FB29DF70>]
type: 1
name: МРЛ2
q: 9.428630395041587
s: 0.015
H_in: 80.0
h: 5.33211146484375
z: 0
p: 1
n: 4
q_out: <function q_out_simple at 0x00000218FB2A5760>
H_add: 0
observer: None
H_out: 74.66788853515625


# Тест рисования графа

In [ ]:
import networkx as nx